<a href="https://colab.research.google.com/github/hyaochn/RUP-Net/blob/master/RUP_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Residual U-Net + Pixel De-convolution Network**



1.   Import library
2.   Mount Google Drive to load dataset. Att: /X, /Y to load correctly.


In [2]:
%tensorflow_version 2.x
import tensorflow as tf
print("tensorflow version: ",tf.__version__)


TensorFlow 2.x selected.
tensorflow version:  2.1.0


In [3]:
from google.colab import drive
drive.mount('gdrive')
#!ls "gdrive/My Drive/Deep-Learning/Data"


!pip install tensorflow_io

import tensorflow_io as tfio
print("tensorflow_io version: ",tfio.__version__)

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).
tensorflow_io version:  0.12.0


In [0]:
root_dir="gdrive/My Drive/Deep-Learning/"
train_fn=root_dir+"Data/train.h5"
test_fn=root_dir+"Data/test.h5"

#!ls "$train_fn"

x_train, y_train= tfio.IODataset.from_hdf5(train_fn, dataset='/X'), tfio.IODataset.from_hdf5(train_fn, dataset='/Y')
x_test, y_test= tfio.IODataset.from_hdf5(test_fn, dataset='/X'), tfio.IODataset.from_hdf5(test_fn, dataset='/Y')

#conf=tf.io.read_file(root_dir+"Configures/train.csv")

In [0]:
#import h5py
#train_f=h5py.File(train_fn,'r')
#test_f=h5py.File(test_fn,'r')
#tf.data.Dataset.from_tensors(train_f)

In [43]:
import numpy as np
#last channel axis is also number of class (1+1=2)
#image_shape=x_train.element_spec.shape
#print(image_shape)
BATCH_SIZE=2
EPOCH=5

SHUFFLE_SIZE=np.prod(x_train.element_spec.shape)
train_set=tf.data.Dataset.zip((x_train, y_train)).batch(BATCH_SIZE).shuffle(SHUFFLE_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_set=tf.data.Dataset.zip((x_test, y_test)).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
xxx=train_set.element_spec
#type(xxx)
print(xxx[0].shape)

(None, 64, 64, 64, 1)


In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(64,64,64,1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2)
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])

In [49]:
model.fit(train_set, epochs=EPOCH)


Epoch 1/5
      1/Unknown - 2s 2s/step

InvalidArgumentError: ignored